# Enhance reproducability of results

Since all ML algorithms are inherently stochastic, running them multiple times give different solutions. However, for debugging and model performance comparison, it is important to have a degree of reproducability in the results.

In [0]:
#%env PYTHONHASHSEED=0 
#%env CUDA_VISIBLE_DEVICES=""

In [0]:
import numpy as np
import tensorflow as tf
import random
import os
#tf.logging.set_verbosity(tf.logging.ERROR)

#np.random.seed(1)
#random.seed(1)
#tf.set_random_seed(1)

#config = tf.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
#from keras import backend as K
#sess = tf.Session(graph=tf.get_default_graph(), config=config)
#K.set_session(sess)

In [0]:
def resetRNG(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.set_random_seed(seed_value)

# Module IMPORTS

In [0]:
#from keras.layers import Input, Dense, Flatten, Reshape, concatenate, Conv2D, Conv1D
#from keras.layers import SeparableConv1D, Lambda, Conv2DTranspose, LeakyReLU
from keras.layers import *
from keras.models import Model, load_model
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K 
#from keras import initializers
import keras

from IPython.display import SVG, display
from keras.utils.vis_utils import plot_model, model_to_dot

# Import DATABASE

Import the .npz Database containing 5SS mechanisms and cplr paths.

In [0]:
def import_DB(dbname):
    current_dir=os.getcwd()
    db_path=os.path.join(current_dir,'..','1. DB Preprocessing',dbname)
    db = np.load(db_path)
    
    return db

# Autoencoder TRAINING

In [0]:
# Train same model architecture multiple times since the training process is stochastic

def AE_analysis(AE_model, iterations, x_train, x_test, filename='model', y_train=None, y_test=None):
    # AE_model: Keras Autoencoder model
    # iterations: Times the autoencoder is trained from scratch. Only best model saved.
    
    val_loss_list=[]
    best_val_loss=np.inf;
    for i in range(iterations):
        print('Model no. '+str(i+1))
        AE, E = AE_model()
        val_loss, train_data=train_AE(AE, E, x_train, x_test, y_train, y_test)
        val_loss_list.append(val_loss)
        if best_val_loss>val_loss:
            best_AE=AE
            best_E=E
            best_train_data=train_data
            best_val_loss=val_loss
    
    # PRINT MEAN, VARIANCE and BEST MODEL METRICS
    print('**************************************************')
    print('Validation Loss Stastics:')
    print('Values: '+str(val_loss_list))
    print('Mean: '+str(np.around(np.mean(val_loss_list), decimals=4)))
    print('Standard Deviation: '+str(np.around(np.std(val_loss_list), decimals=4)))
    print('**************************************************')
    
    # SAVE BEST MODEL
    AEfilename='AE_'+filename+'_val_loss_'+str(best_val_loss)+'.h5'
    AEfilepath=os.path.join('models',AEfilename)
    best_AE.save(AEfilepath)
    
    return best_AE, best_E, best_train_data

def rmse(y_true, y_pred):
        #return K.sqrt(K.mean(K.square(y_pred - y_true))) 
        return K.sqrt(keras.losses.mean_squared_error(y_true, y_pred))

def train_AE(autoencoder, encoder, x_train, x_test, y_train=None, y_test=None, epochs=1000):
    # TRAINING PHASE
    # Losses: mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
    autoencoder.compile(optimizer='adam', loss=rmse, 
                        metrics=['accuracy', 'mse', 'mae', rmse])
    earlyStopping = EarlyStopping(monitor='val_loss', patience=25, verbose=1, mode='min',
                                  restore_best_weights=True)
    if (y_train is None):
      autoencoder_train=autoencoder.fit(x_train, x_train, epochs=epochs, batch_size=128, 
                                        shuffle=False, validation_data=(x_test, x_test), 
                                        verbose=0, callbacks=[earlyStopping])
    else:
      autoencoder_train=autoencoder.fit(x_train, y_train, epochs=epochs, batch_size=128, 
                                        shuffle=False, validation_data=(x_test, y_test), 
                                        verbose=0, callbacks=[earlyStopping])
    
    # TRAINING HISTORY
    val_loss = autoencoder_train.history['val_loss']
    f_val_loss=np.round_(val_loss[-1],decimals=4)
    
    return f_val_loss, autoencoder_train

# Autoencoder VISUALIZATION

In [1]:
def visualize_AE(AE, train_data, x_test, y_test=None):
    # VISUALIZING MODEL ARCHITECTURE
    AE.summary()
    #display(SVG(model_to_dot(AE, show_shapes=True, show_layer_names=False, dpi=65).create(prog='dot', format='svg')))
    
    # VISUALIZING LEARNING CURVES
    visualize_learning_curves(train_data)
    
    # VISUALIZE RECONSTRUCTED CURVES
    reconst_curve = AE.predict(x_test)
    if (y_train is None):
      visualize_reconstructed_CplrCurves(x_test, reconst_curve, 10)
    else:
      visualize_reconstructed_CplrCurves(y_test, reconst_curve, 10)

    
def visualize_learning_curves(train_data):
    loss = train_data.history['loss']
    val_loss = train_data.history['val_loss']

    #f_loss=np.round_(loss[-1],decimals=4)
    #f_val_loss=np.round_(val_loss[-1],decimals=4)
    
    min_epoch=val_loss.index(min(val_loss))
    min_loss=np.round_(loss[min_epoch],decimals=4)
    min_val_loss=np.round_(val_loss[min_epoch],decimals=4)
    
    print('Best Epoch: '+str(min_epoch+1))
    print('Training loss: '+str(min_loss))
    print('Validation loss: '+str(min_val_loss))
    
    epochsRange = range(len(loss))
    plt.figure(figsize=(14, 7))
    plt.plot(epochsRange[4:], loss[4:], 'r', label='Training loss')
    plt.plot(epochsRange[4:], val_loss[4:], 'b', label='Validation loss')
    plt.title('Learning Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Error')
    plt.legend()
    plt.show()

    
def visualize_reconstructed_CplrCurves(x_test, reconst_curve, n=10, b_spline=False):
    fig= plt.figure(figsize=(3*4, 3*n))
    fig.suptitle('Isometric, Top, Front and Right Views', fontsize=14, fontweight='bold')
    for i in range(n):
        # Isometric View
        ax = plt.subplot(n, 4, 4*i+1, projection='3d')
        if b_spline:
            reconst_curve[i]=b_spline_interpolation(reconst_curve[i], 100)
        plotPath3D(reconst_curve[i], ax, 1,'r' )
        plotPath3D(x_test[i], ax, 1)
        
        #x-y, x-z, y-z plane view
        xy_recon=reconst_curve[i,:,0:2]
        xy_orig=x_test[i,:,0:2]
        xz_recon=np.concatenate(([reconst_curve[i,:,0]], [reconst_curve[i,:,2]]),axis=0).T
        xz_orig=np.concatenate(([x_test[i,:,0]], [x_test[i,:,2]]),axis=0).T
        yz_recon=reconst_curve[i,:,1:3]
        yz_orig=x_test[i,:,1:3]
        
        # Top View
        ax = plt.subplot(n, 4, 4*i+2)
        plotPath2D(xy_recon, ax, 1)
        plotPath2D(xy_orig, ax, 1,'r','X coord','Y coord')
        
        # Front View
        ax = plt.subplot(n, 4, 4*i+3)
        plotPath2D(xz_recon, ax, 1)
        plotPath2D(xz_orig, ax, 1,'r','X coord','Z coord')
        
        # Right View
        ax = plt.subplot(n, 4, 4*i+4)
        plotPath2D(yz_recon, ax, 1)
        plotPath2D(yz_orig, ax, 1,'r','Y coord','Z coord')
        
    fig.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()


In [ ]:
def visualize_VAE(AE, x_test, y_test):
    # VISUALIZE RECONSTRUCTED CURVES
    for index in range(10):
        visualize_reconstructed_CplrMotion_VAE(x_test, y_test, AE, n=index)

def visualize_reconstructed_CplrMotion_VAE(x_test, y_test, AE, n=10, n_reconst=5):
    fig= plt.figure(figsize=(3*4, 3*2))
    fig.suptitle('Coupler Motion and its coordinate plots', fontsize=14, fontweight='bold')
    
    inp_cplr=x_test[n]
    out_cplr=y_test[n]
    reconst_cplr=[]
    for j in range(n_reconst):
        reconst_cplr.append(AE.predict(np.expand_dims(inp_cplr, axis=0))[0])
    reconst_cplr=np.array(reconst_cplr)
    out_path=out_cplr[:,0:3]
    out_ori=out_cplr[:,3:8]
    
    # Isometric Coupler Motion View
    ax = plt.subplot(2, 4, 1, projection='3d')
    plotMotion(out_path,out_ori,ax,.5,scale=.1)
    
    # X,Y,Z coord
    ax_x = plt.subplot(2, 4, 2)
    ax_y = plt.subplot(2, 4, 3)
    ax_z = plt.subplot(2, 4, 4)
    visualizeCoord(reconst_cplr[:,:,0], out_cplr[:,0], ax_x, limit=[-1,1])
    visualizeCoord(reconst_cplr[:,:,1], out_cplr[:,1], ax_y, limit=[-1,1])
    visualizeCoord(reconst_cplr[:,:,2], out_cplr[:,2], ax_z, limit=[-1,1])
    
    # Quaternion coord
    ax_qx = plt.subplot(2, 4, 5)
    ax_qy = plt.subplot(2, 4, 6)
    ax_qz = plt.subplot(2, 4, 7)
    ax_qw = plt.subplot(2, 4, 8)
    visualizeCoord(reconst_cplr[:,:,3], out_cplr[:,3], ax_qx, limit=[-1,1])
    visualizeCoord(reconst_cplr[:,:,4], out_cplr[:,4], ax_qy, limit=[-1,1])
    visualizeCoord(reconst_cplr[:,:,5], out_cplr[:,5], ax_qz, limit=[-1,1])
    visualizeCoord(reconst_cplr[:,:,6], out_cplr[:,6], ax_qw, limit=[-1,1])
    
    fig.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

def visualizeCoord(reconst, target, ax, limit=[None,None]):
    x=range(target.shape[0])
    for j in range(reconst.shape[0]):
        ax.plot(x, reconst[j,:], 'gray')
    ax.plot(x, target, 'r')
    ax.set(ylim=limit)

In [0]:
# Plotting Functions

%matplotlib inline
#%matplotlib notebook
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from scipy import interpolate


def plotPath3D(Pts, ax, limit, color = 'gray', x_label='', y_label='', z_label=''):
    xline=Pts[:,0]
    yline=Pts[:,1]
    zline=Pts[:,2]
    ax.plot3D(xline, yline, zline, color)
    ax.auto_scale_xyz([0, limit], [0, limit], [0, limit])
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_zlabel(z_label)
    
def plotPath2D(Pts, ax, limit, color = 'gray', x_label='', y_label=''):
    xline=Pts[:,0]
    yline=Pts[:,1]
    ax.plot(xline, yline, color)
    ax.set(xlim=(0, limit), ylim=(0, limit))
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)

def plotXYZ(center, RotMat, ax):
    C=np.vstack((center,center,center))
    R=RotMat
    r=(1,0,0)
    g=(0,1,0)
    b=(0,0,1)
    ax.quiver(C[:,0], C[:,1], C[:,2], R[:,0], R[:,1], R[:,2],color=(r,g,b,r,r,g,g,b,b))


def b_spline_interpolation(inp_pts, out_n):
    # Fit cubic B-spline to the points
    xp=inp_pts[i,:,0]
    yp=inp_pts[i,:,1]
    zp=inp_pts[i,:,2]
        
    # Check for duplicate points as interpolation routine errors out
    okay = np.where(np.abs(np.diff(xp)) + np.abs(np.diff(yp)) + np.abs(np.diff(zp)) > 0)
    xp = np.r_[xp[okay], xp[-1]]
    yp = np.r_[yp[okay], yp[-1]]
    zp = np.r_[zp[okay], zp[-1]]
    
    tck, u =interpolate.splprep([xp,yp,zp],s=1)
    num_pts=out_n
    u_fine = np.linspace(0,1,num_pts)
    x_f, y_f, z_f = interpolate.splev(u_fine, tck)
    Path_f = np.vstack(([x_100],[y_100],[z_100])).T
    
    return Path_f


# Plotting Functions

def plotPath(Pts, ax, limit, color = 'silver'):
    xline=Pts[:,0]
    yline=Pts[:,1]
    zline=Pts[:,2]
    ax.plot3D(xline, yline, zline, color, linewidth=.8)
    ax.auto_scale_xyz([-limit, limit], [-limit, limit], [-limit, limit])
    
    ax.set_xlabel('X', fontweight ='bold')
    ax.set_ylabel('Y', fontweight ='bold')
    ax.set_zlabel('Z', fontweight ='bold')
    plt.tight_layout()

def plotXYZ(center, RotMat, ax, scale=1):
    C=np.vstack((center,center,center))
    R=scale*RotMat
    r='orangered'
    g='limegreen'
    b='blue'
    ax.quiver(C[:,0], C[:,1], C[:,2], R[:,0], R[:,1], R[:,2],color=(r,g,b,r,r,g,g,b,b),linewidth=.8)
    plt.tight_layout()


def visualizePaths(Paths,rows=10,cols=3, axlimit=.5):
    path_index=0;
    fig = plt.figure(figsize=2*plt.figaspect(rows/cols))
    for i in range(rows):
        for j in range(cols):
            Pts=Paths[path_index]
            path_index=path_index+1
            ax = fig.add_subplot(rows, cols, path_index, projection='3d')
            plotPath(Pts, ax, axlimit)
            #plotXYZ(np.zeros((1, 3)), np.identity(3), ax)
    plt.tight_layout()
    plt.show()

def visualizePathsRand(Paths,rows=10,cols=3, axlimit=.5):
    path_index=0;
    fig = plt.figure(figsize=2*plt.figaspect(rows/cols))
    for i in range(rows):
        for j in range(cols):
            Pts=Paths[np.random.randint(Paths.shape[0])]
            path_index=path_index+1
            ax = fig.add_subplot(rows, cols, path_index, projection='3d')
            plotPath(Pts, ax, axlimit)
            #plotXYZ(np.zeros((1, 3)), np.identity(3), ax)
    plt.tight_layout()
    plt.show()
    
    

def plotMotion(Pts, Orient, ax, limit, color = 'gray', density=1, scale=.1):
    plotPath(Pts, ax, limit, color)
    n=int(len(Pts)*density)
    for i in range(n):
        Rot_i=quat2rot(Orient[int(np.round(i/density))])
        plotXYZ(Pts[int(np.round(i/density))], Rot_i, ax, limit*scale)
    plotXYZ(Pts[-1], quat2rot(Orient[-1]), ax, limit*scale)
    ax.auto_scale_xyz([-limit, limit], [-limit, limit], [-limit, limit])
    
    ax.set_xlabel('X', fontweight ='bold')
    ax.set_ylabel('Y', fontweight ='bold')
    ax.set_zlabel('Z', fontweight ='bold')
    plt.tight_layout()

def quat2rot(Quat):
    q1=Quat[0]
    q2=Quat[1]
    q3=Quat[2]
    q4=Quat[3]
    R11=q1**2-q2**2-q3**2+q4**2
    R12=2*(q1*q2-q3*q4)
    R13=2*(q1*q3+q2*q4)
    R21=2*(q1*q2+q3*q4)
    R22=q4**2-q1**2+q2**2-q3**2
    R23=2*(q2*q3-q1*q4)
    R31=2*(q1*q3-q2*q4)
    R32=2*(q2*q3+q1*q4)
    R33=q4**2-q1**2-q2**2+q3**2
    return np.array([[R11,R12,R13],[R21,R22,R23],[R31,R32,R33]])


def plotLine3D(P1,P2,ax,color):
    ax.plot3D([P1[0],P2[0]], [P1[1],P2[1]], [P1[2],P2[2]],linewidth=3, color=color)

def plotMech5SS(mech,ax):
    Fp1=mech[0]; Fp2=mech[1];Fp3=mech[2]; Fp4=mech[3];Fp5=mech[4];
    Mp1=mech[5]; Mp2=mech[6];Mp3=mech[7]; Mp4=mech[8];Mp5=mech[9];
    C=mech[10];
    
    # Plot Fixed Pivots
    ax.scatter3D(mech[0:5,0],mech[0:5,1],mech[0:5,2],marker=6,linewidths=5,s=200,color='k',alpha=1)
    
    # Plot Dyads
    c1='grey'
    ax.scatter3D(mech[0:5,0],mech[0:5,1],mech[0:5,2],marker='o',s=30,color=c1,alpha=1)
    plotLine3D(Fp1,Mp1,ax,c1)
    plotLine3D(Fp2,Mp2,ax,c1)
    plotLine3D(Fp3,Mp3,ax,c1)
    plotLine3D(Fp4,Mp4,ax,c1)
    plotLine3D(Fp5,Mp5,ax,c1)
    
    # Plot Coupler
    c2='steelblue'
    ax.scatter3D(mech[5:10,0],mech[5:10,1],mech[5:10,2],marker='o',s=40,color=c2,alpha=1)
    plotLine3D(Mp1,C,ax,c2)
    plotLine3D(Mp2,C,ax,c2)
    plotLine3D(Mp3,C,ax,c2)
    plotLine3D(Mp4,C,ax,c2)
    plotLine3D(Mp5,C,ax,c2)
    ax.scatter3D(C[0],C[1],C[2],marker='o',s=40,color='k',alpha=1)
    
    ax.set_xlabel('X', fontweight ='bold')
    ax.set_ylabel('Y', fontweight ='bold')
    ax.set_zlabel('Z', fontweight ='bold')
    plt.tight_layout()


def visualizeMotions(Paths,Orients,rows=10,cols=3, axlimit=.5, density=1):
    path_index=0;
    fig = plt.figure(figsize=2*plt.figaspect(rows/cols))
    for i in range(rows):
        for j in range(cols):
            Pts=Paths[path_index]
            Ori=Orients[path_index]
            path_index=path_index+1
            ax = fig.add_subplot(rows, cols, path_index, projection='3d')
            plotMotion(Pts,Ori, ax, axlimit, density=density)
            #plotXYZ(np.zeros((1, 3)), np.identity(3), ax)
    plt.tight_layout()
    plt.show()
    
def visualizeMotionsRand(Paths,Orients,rows=10,cols=3, axlimit=.5, density=1):
    path_index=0;
    fig = plt.figure(figsize=2*plt.figaspect(rows/cols))
    for i in range(rows):
        for j in range(cols):
            rndInt=np.random.randint(Paths.shape[0])
            Pts=Paths[rndInt]
            Ori=Orients[rndInt]
            path_index=path_index+1
            ax = fig.add_subplot(rows, cols, path_index, projection='3d')
            plotMotion(Pts,Ori, ax, axlimit, density=density)
            plt.title("Index "+str(rndInt))
            #plotXYZ(np.zeros((1, 3)), np.identity(3), ax)
    plt.tight_layout()
    plt.show()

## Visualize variation of coupler curves with change in z-space parameters

# Autoencoder Models: 

In [0]:
######################################################################
# Sigmoid Z_30 hidden_2 [300,100,100,30,100,100,300]
def aeSig():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(100, activation='sigmoid')(x)
    x=Dense(100, activation='sigmoid')(x)
    encoded=Dense(30, activation='sigmoid')(x)

    # Decoding
    x=Dense(100, activation='sigmoid')(encoded)
    x=Dense(100, activation='sigmoid')(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# Tanh Z_30 hidden_2 [300,100,100,30,100,100,300]
def aeTanh():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(100, activation='tanh')(x)
    x=Dense(100, activation='tanh')(x)
    encoded=Dense(30, activation='tanh')(x)

    # Decoding
    x=Dense(100, activation='tanh')(encoded)
    x=Dense(100, activation='tanh')(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# RELU Z_30 hidden_2 [300,100,100,30,100,100,300]
def aeRELU():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    encoded=Dense(30, activation='relu')(x)

    # Decoding
    x=Dense(100, activation='relu')(encoded)
    x=Dense(100, activation='relu')(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# LRELU Z_30 hidden_2 [300,100,100,30,100,100,300]
def aeLRELU():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(100)(x)
    x=LeakyReLU(alpha=0.01)(x)
    x=Dense(100)(x)
    x=LeakyReLU(alpha=0.01)(x)
    x=Dense(30)(x)
    encoded=LeakyReLU(alpha=0.01)(x)

    # Decoding
    x=Dense(100)(encoded)
    x=LeakyReLU(alpha=0.01)(x)
    x=Dense(100)(x)
    x=LeakyReLU(alpha=0.01)(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################

In [0]:
######################################################################
# RELU Z_2 hidden_3 [300,150,100,70,2,70,100,150,300]
def aeZ2():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(150, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(70, activation='relu')(x)
    encoded=Dense(2, activation='relu')(x)

    # Decoding
    x=Dense(70, activation='relu')(encoded)
    x=Dense(100, activation='relu')(x)
    x=Dense(150, activation='relu')(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# RELU Z_3 hidden_3 [300,150,100,70,3,70,100,150,300]
def aeZ3():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(150, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(70, activation='relu')(x)
    encoded=Dense(3, activation='relu')(x)

    # Decoding
    x=Dense(70, activation='relu')(encoded)
    x=Dense(100, activation='relu')(x)
    x=Dense(150, activation='relu')(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# RELU Z_10 hidden_3 [300,150,100,70,10,70,100,150,300]
def aeZ10():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(150, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(70, activation='relu')(x)
    encoded=Dense(10, activation='relu')(x)

    # Decoding
    x=Dense(70, activation='relu')(encoded)
    x=Dense(100, activation='relu')(x)
    x=Dense(150, activation='relu')(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# RELU Z_15 hidden_3 [300,150,100,70,15,70,100,150,300]
def aeZ15():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(150, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(70, activation='relu')(x)
    encoded=Dense(15, activation='relu')(x)

    # Decoding
    x=Dense(70, activation='relu')(encoded)
    x=Dense(100, activation='relu')(x)
    x=Dense(150, activation='relu')(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# RELU Z_30 hidden_3 [300,150,100,70,30,70,100,150,300]
def aeZ30():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(150, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(70, activation='relu')(x)
    encoded=Dense(30, activation='relu')(x)

    # Decoding
    x=Dense(70, activation='relu')(encoded)
    x=Dense(100, activation='relu')(x)
    x=Dense(150, activation='relu')(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# RELU Z_60 hidden_3 [300,150,100,70,60,70,100,150,300]
def aeZ60():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(150, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(70, activation='relu')(x)
    encoded=Dense(60, activation='relu')(x)

    # Decoding
    x=Dense(70, activation='relu')(encoded)
    x=Dense(100, activation='relu')(x)
    x=Dense(150, activation='relu')(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# RELU Z_300 hidden_3 [300,150,100,70,300,70,100,150,300]
def aeZ300():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(150, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(70, activation='relu')(x)
    encoded=Dense(300, activation='relu')(x)

    # Decoding
    x=Dense(70, activation='relu')(encoded)
    x=Dense(100, activation='relu')(x)
    x=Dense(150, activation='relu')(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder
    
######################################################################

In [0]:
######################################################################
# RELU Z_30 hidden_0 [300,30,300]
def aeDepth0():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    encoded=Dense(30, activation='relu')(x)

    # Decoding
    x=Dense(300)(encoded)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# RELU Z_30 hidden_1 [300,200,30,200,300]
def aeDepth1():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(200, activation='relu')(x)
    encoded=Dense(30, activation='relu')(x)

    # Decoding
    x=Dense(200, activation='relu')(encoded)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# RELU Z_30 hidden_2 [300,200,150,30,150,200,300]
def aeDepth2():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(200, activation='relu')(x)
    x=Dense(150, activation='relu')(x)
    encoded=Dense(30, activation='relu')(x)

    # Decoding
    x=Dense(150, activation='relu')(encoded)
    x=Dense(200, activation='relu')(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# RELU Z_30 hidden_3 [300,200,150,100,30,100,150,200,300]
def aeDepth3():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(200, activation='relu')(x)
    x=Dense(150, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    encoded=Dense(30, activation='relu')(x)

    # Decoding
    x=Dense(100, activation='relu')(encoded)
    x=Dense(150, activation='relu')(x)
    x=Dense(200, activation='relu')(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# RELU Z_30 hidden_4 [300,200,150,100,75,30,75,100,150,200,300]
def aeDepth4():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(200, activation='relu')(x)
    x=Dense(150, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(75, activation='relu')(x)
    encoded=Dense(30, activation='relu')(x)

    # Decoding
    x=Dense(75, activation='relu')(encoded)
    x=Dense(100, activation='relu')(x)
    x=Dense(150, activation='relu')(x)
    x=Dense(200, activation='relu')(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# RELU Z_30 hidden_5 [300,200,150,100,75,50,30,50,75,100,150,200,300]
def aeDepth5():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(200, activation='relu')(x)
    x=Dense(150, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(75, activation='relu')(x)
    x=Dense(50, activation='relu')(x)
    encoded=Dense(30, activation='relu')(x)

    # Decoding
    x=Dense(50, activation='relu')(encoded)
    x=Dense(75, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(150, activation='relu')(x)
    x=Dense(200, activation='relu')(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################

In [0]:
######################################################################
# Regularized RELU Z_3 hidden_3 [300,150,100,70,3,70,100,150,300]
def aeReg():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(150, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(70, activation='relu')(x)
    encoded=Dense(3, activation='relu',activity_regularizer=regularizers.l1(10e-7))(x)

    # Decoding
    x=Dense(70, activation='relu')(encoded)
    x=Dense(100, activation='relu')(x)
    x=Dense(150, activation='relu')(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################

# Convolutional Autoencoder Models: 

## Conv 1D

In [0]:
######################################################################
# Cov1D Layer=1, Dense Layer=0, Filter=Global
def ae1DCov1_D0_FG():
    inp_curve=Input(shape=(100,3))

    # Encoding
    x= Reshape((100, 3, 1))(inp_curve)
    x= Conv2D(100, (100,3), activation='relu')(x)
    x=Flatten()(x)
    encoded=Dense(30, activation='relu')(x)
    
    # Decoding
    x=Dense(100, activation='relu')(encoded)
    x=Reshape((1, 1, 100))(x)
    x=Conv2DTranspose(1, (100,3))(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# Cov1D Layer=3, Dense Layer=0, Filter=Global
def ae1DCov3_D0_FG():
    inp_curve=Input(shape=(100,3))

    # Encoding
    x= Reshape((100, 3, 1))(inp_curve)
    x= Conv2D(200, (100,3), activation='relu')(x)
    x= Reshape((200, 1, 1))(x)
    x= Conv2D(150, (200,1), activation='relu')(x)
    x= Reshape((150, 1, 1))(x)
    x= Conv2D(100, (150,1), activation='relu')(x)
    x= Flatten()(x)
    encoded=Dense(30, activation='relu')(x)
    
    # Decoding
    x= Dense(100, activation='relu')(encoded)
    x= Reshape((1, 1, 100))(x)
    x= Conv2DTranspose(1, (150,1), activation='relu')(x)
    x= Reshape((1, 1, 150))(x)
    x= Conv2DTranspose(1, (200,1), activation='relu')(x)
    x= Reshape((1, 1, 200))(x)
    x=Conv2DTranspose(1, (100,3))(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# Cov1D Layer=1, Dense Layer=3, Filter=Global
def ae1DCov1_D3_FG():
    inp_curve=Input(shape=(100,3))

    # Encoding
    x= Reshape((100, 3, 1))(inp_curve)
    x= Conv2D(100, (100,3), activation='relu')(x)
    x=Flatten()(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    encoded=Dense(30, activation='relu')(x)
    
    # Decoding
    x=Dense(100, activation='relu')(encoded)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Reshape((1, 1, 100))(x)
    x=Conv2DTranspose(1, (100,3))(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder
######################################################################
# Cov1D Layer=3, Dense Layer=3, Filter=Global
def ae1DCov3_D3_FG():
    inp_curve=Input(shape=(100,3))

    # Encoding
    x= Reshape((100, 3, 1))(inp_curve)
    x= Conv2D(200, (100,3), activation='relu')(x)
    x= Reshape((200, 1, 1))(x)
    x= Conv2D(150, (200,1), activation='relu')(x)
    x= Reshape((150, 1, 1))(x)
    x= Conv2D(100, (150,1), activation='relu')(x)
    x= Flatten()(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    encoded=Dense(30, activation='relu')(x)
    
    # Decoding
    x=Dense(100, activation='relu')(encoded)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x= Dense(100, activation='relu')(x)
    x= Reshape((1, 1, 100))(x)
    x= Conv2DTranspose(1, (150,1), activation='relu')(x)
    x= Reshape((1, 1, 150))(x)
    x= Conv2DTranspose(1, (200,1), activation='relu')(x)
    x= Reshape((1, 1, 200))(x)
    x=Conv2DTranspose(1, (100,3))(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder
######################################################################

In [0]:
######################################################################
# Cov1D Layer=1, Dense Layer=0, Filter=Local
def ae1DCov1_D0_FL():
    inp_curve=Input(shape=(100,3))

    # Encoding
    x= Reshape((100, 3, 1))(inp_curve)
    x= Conv2D(10, (10,3), strides=(10, 1), activation='relu')(x)
    x=Flatten()(x)
    encoded=Dense(30, activation='relu')(x)
    
    # Decoding
    x=Dense(100, activation='relu')(encoded)
    x=Reshape((10, 1, 10))(x)
    x=Conv2DTranspose(1, (10,3), strides=(10, 1))(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# Cov1D Layer=3, Dense Layer=0, Filter=Local
def ae1DCov3_D0_FL():
    inp_curve=Input(shape=(100,3))

    # Encoding
    x= Reshape((100, 3, 1))(inp_curve)
    x= Conv2D(20, (10,3), strides=(10, 1), activation='relu')(x)
    x= Conv2D(30, (2,1), strides=(2, 1), activation='relu')(x)
    x= Conv2D(50, (4,1), activation='relu')(x)
    x=Flatten()(x)
    encoded=Dense(30, activation='relu')(x)
    
    # Decoding
    x=Dense(100, activation='relu')(encoded)
    x=Reshape((2, 1, 50))(x)
    x=Conv2DTranspose(30, (4,1), activation='relu')(x)
    x=Conv2DTranspose(20, (2,1), strides=(2, 1), activation='relu')(x)
    x=Conv2DTranspose(1, (10,3), strides=(10, 1))(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# Cov1D Layer=1, Dense Layer=3, Filter=Local
def ae1DCov1_D3_FL():
    inp_curve=Input(shape=(100,3))

    # Encoding
    x= Reshape((100, 3, 1))(inp_curve)
    x= Conv2D(10, (10,3), strides=(10, 1), activation='relu')(x)
    x=Flatten()(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    encoded=Dense(30, activation='relu')(x)
    
    # Decoding
    x=Dense(100, activation='relu')(encoded)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Reshape((10, 1, 10))(x)
    x=Conv2DTranspose(1, (10,3), strides=(10, 1))(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# Cov1D Layer=3, Dense Layer=3, Filter=Local
def ae1DCov3_D3_FL():
    inp_curve=Input(shape=(100,3))

    # Encoding
    x= Reshape((100, 3, 1))(inp_curve)
    x= Conv2D(20, (10,3), strides=(10, 1), activation='relu')(x)
    x= Conv2D(30, (2,1), strides=(2, 1), activation='relu')(x)
    x= Conv2D(50, (4,1), activation='relu')(x)
    x=Flatten()(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    encoded=Dense(30, activation='relu')(x)
    
    # Decoding
    x=Dense(100, activation='relu')(encoded)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Reshape((2, 1, 50))(x)
    x=Conv2DTranspose(30, (4,1), activation='relu')(x)
    x=Conv2DTranspose(20, (2,1), strides=(2, 1), activation='relu')(x)
    x=Conv2DTranspose(1, (10,3), strides=(10, 1))(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################

## Conv 2D

In [0]:
######################################################################
# Cov2D Layer=1, Dense Layer=0, Filter=Global
def ae2DCov1_D0_FG():
    inp_curve=Input(shape=(100,3))

    # Encoding
    x= Reshape((100, 3, 1))(inp_curve)
    x= Conv2D(50, (100,1), activation='relu')(x)
    x=Flatten()(x)
    encoded=Dense(30, activation='relu')(x)
    
    # Decoding
    x=Dense(150, activation='relu')(encoded)
    x=Reshape((1, 3, 50))(x)
    x=Conv2DTranspose(1, (100,1))(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# Cov2D Layer=3, Dense Layer=0, Filter=Global
def ae2DCov3_D0_FG():
    inp_curve=Input(shape=(100,3))

    # Encoding
    x= Reshape((100, 3, 1))(inp_curve)
    x= Conv2D(70, (100,1), activation='relu')(x)
    x= Reshape((70, 3, 1))(x)
    x= Conv2D(50, (70,1), activation='relu')(x)
    x= Reshape((50, 3, 1))(x)
    x= Conv2D(30, (50,1), activation='relu')(x)
    x= Flatten()(x)
    encoded=Dense(30, activation='relu')(x)
    
    # Decoding
    x= Dense(90, activation='relu')(encoded)
    x= Reshape((1, 3, 30))(x)
    x= Conv2DTranspose(1, (50,1), activation='relu')(x)
    x= Reshape((1, 3, 50))(x)
    x= Conv2DTranspose(1, (70,1), activation='relu')(x)
    x= Reshape((1, 3, 70))(x)
    x=Conv2DTranspose(1, (100,1))(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# Cov2D Layer=1, Dense Layer=3, Filter=Global
def ae2DCov1_D3_FG():
    inp_curve=Input(shape=(100,3))

    # Encoding
    x= Reshape((100, 3, 1))(inp_curve)
    x= Conv2D(50, (100,1), activation='relu')(x)
    x=Flatten()(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    encoded=Dense(30, activation='relu')(x)
    
    # Decoding
    x=Dense(100, activation='relu')(encoded)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(150, activation='relu')(x)
    x=Reshape((1, 3, 50))(x)
    x=Conv2DTranspose(1, (100,1))(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder
######################################################################
# Cov2D Layer=3, Dense Layer=3, Filter=Global
def ae2DCov3_D3_FG():
    inp_curve=Input(shape=(100,3))

    # Encoding
    x= Reshape((100, 3, 1))(inp_curve)
    x= Conv2D(70, (100,1), activation='relu')(x)
    x= Reshape((70, 3, 1))(x)
    x= Conv2D(50, (70,1), activation='relu')(x)
    x= Reshape((50, 3, 1))(x)
    x= Conv2D(30, (50,1), activation='relu')(x)
    x= Flatten()(x)
    x= Dense(90, activation='relu')(x)
    x= Dense(90, activation='relu')(x)
    x= Dense(90, activation='relu')(x)
    encoded=Dense(30, activation='relu')(x)
    
    # Decoding
    x= Dense(90, activation='relu')(encoded)
    x= Dense(90, activation='relu')(x)
    x= Dense(90, activation='relu')(x)
    x= Dense(90, activation='relu')(x)
    x= Reshape((1, 3, 30))(x)
    x= Conv2DTranspose(1, (50,1), activation='relu')(x)
    x= Reshape((1, 3, 50))(x)
    x= Conv2DTranspose(1, (70,1), activation='relu')(x)
    x= Reshape((1, 3, 70))(x)
    x=Conv2DTranspose(1, (100,1))(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder
######################################################################

In [0]:
######################################################################
# Cov2D Layer=1, Dense Layer=0, Filter=Local
def ae2DCov1_D0_FL():
    inp_curve=Input(shape=(100,3))

    # Encoding
    x= Reshape((100, 3, 1))(inp_curve)
    x= Conv2D(5, (10,1), strides=(10, 1), activation='relu')(x)
    x=Flatten()(x)
    encoded=Dense(30, activation='relu')(x)
    
    # Decoding
    x=Dense(150, activation='relu')(encoded)
    x=Reshape((10, 3, 5))(x)
    x=Conv2DTranspose(1, (10,1), strides=(10, 1))(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# Cov2D Layer=3, Dense Layer=0, Filter=Local
def ae2DCov3_D0_FL():
    inp_curve=Input(shape=(100,3))

    # Encoding
    x= Reshape((100, 3, 1))(inp_curve)
    x= Conv2D(15, (10,1), strides=(10, 1), activation='relu')(x)
    x= Conv2D(20, (2,1), strides=(2, 1), activation='relu')(x)
    x= Conv2D(25, (4,1), activation='relu')(x)
    x= Flatten()(x)
    encoded=Dense(30, activation='relu')(x)
    
    # Decoding
    x= Dense(150, activation='relu')(encoded)
    x= Reshape((2, 3, 25))(x)
    x= Conv2DTranspose(20, (4,1), activation='relu')(x)
    x= Conv2DTranspose(15, (2,1), strides=(2, 1), activation='relu')(x)
    x= Conv2DTranspose(1, (10,1), strides=(10, 1))(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# Cov2D Layer=1, Dense Layer=3, Filter=Local
def ae2DCov1_D3_FL():
    inp_curve=Input(shape=(100,3))

    # Encoding
    x= Reshape((100, 3, 1))(inp_curve)
    x= Conv2D(5, (10,1), strides=(10, 1), activation='relu')(x)
    x=Flatten()(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    encoded=Dense(30, activation='relu')(x)
    
    # Decoding
    x=Dense(100, activation='relu')(encoded)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    x=Dense(150, activation='relu')(x)
    x=Reshape((10, 3, 5))(x)
    x=Conv2DTranspose(1, (10,1), strides=(10, 1))(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder
######################################################################
# Cov2D Layer=3, Dense Layer=3, Filter=Local
def ae2DCov3_D3_FL():
    inp_curve=Input(shape=(100,3))

    # Encoding
    x= Reshape((100, 3, 1))(inp_curve)
    x= Conv2D(15, (10,1), strides=(10, 1), activation='relu')(x)
    x= Conv2D(20, (2,1), strides=(2, 1), activation='relu')(x)
    x= Conv2D(25, (4,1), activation='relu')(x)
    x= Flatten()(x)
    x= Dense(90, activation='relu')(x)
    x= Dense(90, activation='relu')(x)
    x= Dense(90, activation='relu')(x)
    encoded=Dense(30, activation='relu')(x)
    
    # Decoding
    x= Dense(90, activation='relu')(encoded)
    x= Dense(90, activation='relu')(x)
    x= Dense(90, activation='relu')(x)
    x= Dense(150, activation='relu')(x)
    x= Reshape((2, 3, 25))(x)
    x= Conv2DTranspose(20, (4,1), activation='relu')(x)
    x= Conv2DTranspose(15, (2,1), strides=(2, 1), activation='relu')(x)
    x= Conv2DTranspose(1, (10,1), strides=(10, 1))(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder
######################################################################

## Custom Conv

In [0]:
######################################################################
def aeCov_custom():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x= Reshape((100, 3, 1))(inp_curve)
    x1=Conv2D(4,(5,3), activation='relu')(x)
    x1=MaxPooling2D((2,1))(x1)
    x1=Flatten()(x1)
    x2=Conv2D(4,(20,3), activation='relu')(x)
    x2=MaxPooling2D((2,1))(x2)
    x2=Flatten()(x2)
    x3=Conv2D(4,(50,3), activation='relu')(x)
    x3=MaxPooling2D((2,1))(x3)
    x3=Flatten()(x3)
    x=concatenate([x1,x2,x3], axis=1)
    x=Dense(100, activation='relu')(x)
    encoded=Dense(30, activation='relu')(x)

    # Decoding
    x=Dense(100, activation='relu')(encoded)
    x=Dense(452, activation='relu')(x)
    x=UpSampling1D(2)(x)
    x=Dense(300,activation='relu')(x)
    x=Reshape((300, 1, 1))(x)
    x1=Conv2D(4,(5,1),padding='same', activation='relu')(x)
    x2=Conv2D(4,(20,1),padding='same', activation='relu')(x)
    x3=Conv2D(4,(50,1),padding='same', activation='relu')(x)
    x=Add()([x1, x2, x3])
    x=Conv2D(1,(5,1),padding='same')(x)
    decoded=Reshape((100,3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder
    ######################################################################

## Pure FFT

In [0]:
def Fourier(x):
    #return tf.cast(tf.spectral.fft(tf.cast(x,dtype=tf.complex64)),dtype=tf.float32)
    FDs=tf.spectral.rfft(x)
    re=tf.cast(tf.real(FDs),dtype=tf.float32)
    im=tf.cast(tf.imag(FDs),dtype=tf.float32)
    return tf.concat([re,im],-1)

def invFourier(x):
    #return tf.cast(tf.spectral.ifft(tf.cast(x,dtype=tf.complex64)),dtype=tf.float32)
    re,im=tf.split(x,2,-1)
    FDs=tf.complex(re,im)
    curve=tf.spectral.irfft(FDs)
    return curve

def aeFFT():
    inp_curve=Input(shape=(100,3))

    # Encoding
    x= Permute((2, 1))(inp_curve)
    x = Lambda(Fourier)(x)
    x=Flatten()(x)
    x=Dense(150, activation='relu')(x)
    encoded=Dense(30, activation='relu')(x)

    # Decoding
    x=Dense(150, activation='relu')(encoded)
    x=Dense(306)(x)
    x=Reshape((3, 102))(x)
    x = Lambda(invFourier)(x)
    decoded= Permute((2, 1))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

# VAE Models: